In [59]:
import pandas as pd
import numpy as np
import talib as ta
from plotly import graph_objects as go

# Exponential Moving Average (EMA) Trading Strategy

## Overview
This notebook implements a trading strategy using Exponential Moving Averages (EMA). The strategy generates buy and sell signals based on the crossover of short-term (12-period) and medium-term (128-period) EMAs, with a long-term (200-period) EMA for trend confirmation.

## Strategy Logic:
- **Buy Signal**: When EMA 12 crosses above EMA 128
- **Sell Signal**: When EMA 12 crosses below EMA 128
- **Trend Confirmation**: EMA 200 helps confirm the overall market trend

In [60]:
df = pd.read_csv('../../../data/stocks/AAPL.csv')
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.513393,0.515625,0.513393,0.513393,0.406782,117258400
1,1980-12-15,0.488839,0.488839,0.486607,0.486607,0.385558,43971200
2,1980-12-16,0.453125,0.453125,0.450893,0.450893,0.357260,26432000
3,1980-12-17,0.462054,0.464286,0.462054,0.462054,0.366103,21610400
4,1980-12-18,0.475446,0.477679,0.475446,0.475446,0.376715,18362400
...,...,...,...,...,...,...,...
9904,2020-03-26,246.520004,258.679993,246.360001,258.440002,258.440002,63021800
9905,2020-03-27,252.750000,255.869995,247.050003,247.740005,247.740005,51054200
9906,2020-03-30,250.740005,255.520004,249.399994,254.809998,254.809998,41994100
9907,2020-03-31,255.600006,262.489990,252.000000,254.289993,254.289993,49250500


In [61]:
df['EMA_12'] = ta.EMA(df['Close'], timeperiod=12)
df['EMA_128'] = ta.EMA(df['Close'], timeperiod=128)
df['EMA_200'] = ta.EMA(df['Close'], timeperiod=200)
df

,Date,Open,High,Low,Close,Adj Close,Volume,EMA_12,EMA_128,EMA_200
0,1980-12-12,0.513393,0.515625,0.513393,0.513393,0.406782,117258400,NaN,NaN,NaN
1,1980-12-15,0.488839,0.488839,0.486607,0.486607,0.385558,43971200,NaN,NaN,NaN
2,1980-12-16,0.453125,0.453125,0.450893,0.450893,0.357260,26432000,NaN,NaN,NaN
3,1980-12-17,0.462054,0.464286,0.462054,0.462054,0.366103,21610400,NaN,NaN,NaN
4,1980-12-18,0.475446,0.477679,0.475446,0.475446,0.376715,18362400,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
9904,2020-03-26,246.520004,258.679993,246.360001,258.440002,258.440002,63021800,252.629798,270.570261,256.674214
9905,2020-03-27,252.750000,255.869995,247.050003,247.740005,247.740005,51054200,251.877522,270.216304,256.585317
9906,2020-03-30,250.740005,255.520004,249.399994,254.809998,254.809998,41994100,252.328672,269.977446,256.567652
9907,2020-03-31,255.600006,262.489990,252.000000,254.289993,254.289993,49250500,252.630414,269.734230,256.544988


In [62]:
# Detect crossover events by comparing current and previous values
# Buy: EMA 12 moves from ≤ to > EMA 128 (bullish crossover)
buy_condition = (df['EMA_12'] > df['EMA_128']) & (df['EMA_12'].shift(1) <= df['EMA_128'].shift(1))
# Sell: EMA 12 moves from ≥ to < EMA 128 (bearish crossover)
sell_condition = (df['EMA_12'] < df['EMA_128']) & (df['EMA_12'].shift(1) >= df['EMA_128'].shift(1))

# Create signal columns: 1 for buy, -1 for sell, 0 for no signal
df['Buy_Signal'] = np.where(buy_condition, 1, 0)
df['Sell_Signal'] = np.where(sell_condition, -1, 0)
df

,Date,Open,High,Low,Close,Adj Close,Volume,EMA_12,EMA_128,EMA_200,Buy_Signal,Sell_Signal
0,1980-12-12,0.513393,0.515625,0.513393,0.513393,0.406782,117258400,NaN,NaN,NaN,0,0
1,1980-12-15,0.488839,0.488839,0.486607,0.486607,0.385558,43971200,NaN,NaN,NaN,0,0
2,1980-12-16,0.453125,0.453125,0.450893,0.450893,0.357260,26432000,NaN,NaN,NaN,0,0
3,1980-12-17,0.462054,0.464286,0.462054,0.462054,0.366103,21610400,NaN,NaN,NaN,0,0
4,1980-12-18,0.475446,0.477679,0.475446,0.475446,0.376715,18362400,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9904,2020-03-26,246.520004,258.679993,246.360001,258.440002,258.440002,63021800,252.629798,270.570261,256.674214,0,0
9905,2020-03-27,252.750000,255.869995,247.050003,247.740005,247.740005,51054200,251.877522,270.216304,256.585317,0,0
9906,2020-03-30,250.740005,255.520004,249.399994,254.809998,254.809998,41994100,252.328672,269.977446,256.567652,0,0
9907,2020-03-31,255.600006,262.489990,252.000000,254.289993,254.289993,49250500,252.630414,269.734230,256.544988,0,0


In [63]:
def plot_ema_signals(df: pd.DataFrame) -> go.Figure:
    # Create figure with candlestick chart for price action
    fig = go.Figure(
        go.Candlestick(
            x=df['Date'],
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close']
        )
    )

    # Add price and EMA traces
    fig.add_trace(go.Scatter(x=df['Date'], y=df['Close'], mode='lines', name='Close Price'))
    fig.add_trace(go.Scatter(x=df['Date'], y=df['EMA_12'], mode='lines', name='EMA 12'))
    fig.add_trace(go.Scatter(x=df['Date'], y=df['EMA_128'], mode='lines', name='EMA 128'))
    fig.add_trace(go.Scatter(x=df['Date'], y=df['EMA_200'], mode='lines', name='EMA 200'))

    # Extract buy and sell signal points
    buy_signals = df[df['Buy_Signal'] == 1]
    sell_signals = df[df['Sell_Signal'] == -1]
    
    # Add buy signals as green triangles
    fig.add_trace(go.Scatter(x=buy_signals['Date'], y=buy_signals['Close'], mode='markers', name='Buy Signal', marker=dict(color='green', size=20, symbol='triangle-up')))
    # Add sell signals as red triangles
    fig.add_trace(go.Scatter(x=sell_signals['Date'], y=sell_signals['Close'], mode='markers', name='Sell Signal', marker=dict(color='red', size=20, symbol='triangle-down')))

    fig.update_layout(
        title='Exponential Moving Average (EMA) with Buy/Sell Signals',
        xaxis_rangeslider_visible=False,
        xaxis_title='Date',
        yaxis_title='Price',
        template='plotly_dark' 
    )
    
    return fig

In [64]:
plot = plot_ema_signals(df)
plot.show()

### Backtesting EMA strategy

In [65]:
# Initialize trading parameters
initial_capital = 10000
position = 0  # Number of shares held
capital = initial_capital  # Cash available for trading
bought_and_held_capital = capital  # Benchmark strategy: buy and hold
bought_and_held_shares = capital / df['Close'].iloc[0]  # Calculate buy-and-hold shares at start

# Simulate trading strategy over all days
for i in range(len(df)):
    # BUY LOGIC: If buy signal generated and we have cash available
    if df['Buy_Signal'].iloc[i] == 1 and capital > 0:
        shares_to_buy = capital // df['Close'].iloc[i]  # Calculate shares we can afford (integer division)
        position += shares_to_buy  # Add shares to position
        capital -= shares_to_buy * df['Close'].iloc[i]  # Deduct cost from cash
    
    # SELL LOGIC: If sell signal generated and we're holding shares
    elif df['Sell_Signal'].iloc[i] == -1 and position > 0:
        capital += position * df['Close'].iloc[i]  # Sell all shares at current price
        position = 0  # Close position

    # UPDATE BUY-AND-HOLD BENCHMARK: Track portfolio value if we simply held the stock
    bought_and_held_capital = bought_and_held_shares * df['Close'].iloc[i]

# Calculate final portfolio values
algorithm_final_value = capital + position * df['Close'].iloc[-1]  # Cash + remaining shares value
bought_and_held_final_value = bought_and_held_capital
algorithm_final_percentage = ((algorithm_final_value - initial_capital) / initial_capital) * 100
bought_and_held_final_percentage = ((bought_and_held_final_value - initial_capital) / initial_capital) * 100

# Display results comparing strategy to buy-and-hold benchmark
print(f"Algorithm Final Value: ${algorithm_final_value:.2f} ({algorithm_final_percentage:.2f}%)")
print(f"Buy and Hold Final Value: ${bought_and_held_final_value:.2f} ({bought_and_held_final_percentage:.2f}%)")

Algorithm Final Value: $2279181.27 (22691.81%)
Buy and Hold Final Value: $4692507.82 (46825.08%)
